## Code for Urdu to Roman (as found in other repository)

In [1]:
urdu_lang_dict = {'ا': ['a', 'aa', 'a'], 'ب': ['b'], 'پ': ['p'], 'ت': ['t'], 'ٹ': ['t'], 'ث': ['s'], 'ج': ['j'], 'چ': ['ch'], 'ح': ['h'], 'خ': ['kh'], 'د': ['d'], 'ڈ': ['dd'], 'ذ': ['z'], 'ر': ['r'], 'ڑ': ['rr'], 'ز': ['z'], 'ژ': ['zh'], 'س': ['s'], 'ش': ['sh'], 'ص': ['s'], 'ض': ['z'], 'ط': ['t'], 'ظ': ['z'], 'ع': ['a', "a'", 'o'], 'غ': ['gh'], 'ف': ['f'], 'ق': ['q'], 'ک': ['k'], 'گ': ['g'], 'ل': ['l'], 'م': ['m'], 'ن': ['n'], 'و': ['wo', 'o', 'o'], 'ہ': ['h', 'h', 'eh'], 'ھ': ['h'], 'ه' : ['h'], 'ی': ['y', 'ei', 'i'], 'ئ': ['i'], 'ي': ['e'], 'ے': ['ay'], 'آ': ['aa'], 'ں': ['n'], '؟': ['?'], '،': [','], '.': ['.'], '۔': ['.'], 'ۂ':['a'], 'ء':['a'], 'ِ':[' e']}
no_As = ['ی', 'ے', 'آ', 'ا', 'و', 'ع', 'ہ', 'ں', 'ھ', 'ئ', 'ن']
next_skippers = ['ئی']

def get_position(text, index):
    if index == 0:
        return 0
    elif index == len(text) - 1:
        return 2
    else:
        if text[index - 1] == ' ':
            return 0
        elif text[index + 1] == ' ' or text[index + 1] == '.' or text[index + 1] == '۔' or text[index + 1] == '،' or text[index + 1] == '"':
            return 2
        else:
           return 1


def urdu2roman(input_text):
    skip = False
    output_text = ""
    for i in range(len(input_text)):
        if not skip:
            char = input_text[i]
            if char in urdu_lang_dict:
                pos = get_position(input_text, i)
                if len(urdu_lang_dict[char]) > 1:
                    output_text += urdu_lang_dict[char][pos]
                else:
                    output_text += urdu_lang_dict[char][0]
                if i < len(input_text) - 1:
                    if input_text[i+1] not in no_As and input_text[i] not in no_As:
                        if pos == 1:
                            output_text += 'a'
                    if char+input_text[i+1] in next_skippers:
                        skip = True
                        continue
            else:
                output_text += char
        else:
            skip = False
    return output_text

# N-gram generic class

In [2]:
from collections import defaultdict
from spacy import blank
import random

gram_map = {0: 'uni', 1: 'bi', 2: 'tri'}

tokenizer = blank('ur')

class n_gram_model():
    def __init__(self, n, rev=False):
        self.n = n
        self.vocab = []
        self.gram = [defaultdict(lambda: 0) for i in range(n)]
        self.len = 0
        mul = n*(n+1)/2
        self.lambdas = [(i+1)/mul for i in range(n)]
        self.rev = rev

    def train(self, corpus):
        # calculating uni gram, and populating vocabulary
        for line in corpus:
            # words = ['<s>'] + list(tokenizer(line)) + ['</s>']
            words = list(tokenizer(line))
            for word in words:
                self.gram[0][str(word)] += 1
                self.len += 1
        self.vocab = list(self.gram[0].keys())

        # calculating 2 -> n grams
        for i in range(1, self.n):
            for line in corpus:
                # words = ['<s>'] + list(tokenizer(line)) + ['</s>']
                words = list(tokenizer(line))
                for j in range(0, len(words)-i):
                    grams = ' '.join([str(word) for word in words[j:j+i+1]])
                    self.gram[i][grams] += 1

    def probability(self, word): #unused, used the below function instead
        begin_word = ' '.join(word.split(' ')[:-1])
        print(begin_word, word)
        return (self.gram[-1][word] + 1)/(self.gram[-2][begin_word] + len(self.vocab))

    def interpolated_probability(self, word):
        prob = 0
        spl_word = word.split(' ')
        for i in range(1, self.n):
            # print(f'Lambda {self.lambdas[-i]} for word {spl_word[:len(spl_word) - i + 1]} small {spl_word[:len(spl_word) - i]}')
            prob += self.lambdas[-i] * (self.gram[-i][' '.join(spl_word[:len(spl_word) - i + 1])] + 1)/(
                self.gram[-i - 1][' '.join(spl_word[:len(spl_word) - i])] + len(self.vocab))
            # print(f'prob {prob}')
        prob += self.lambdas[0] * (self.gram[0]
                                   [spl_word[0]] + 1)/(self.len + len(self.vocab))
        return prob

    def generate_poetry(self):
        if self.n == 1:
            return 'Unigram poetry generation not supported.'
        poetry = ''
        word_s = random.choice(list(self.gram[-2].keys()))
        for i in range(7):  # stanzas
            for j in range(2):  # verses
                poetry += word_s + ' '
                word_count = random.randint(6, 9)
                k = 0
                while k < word_count:  # words per verse
                    candidates = []
                    possibilites = list(self.gram[-1].keys())
                    for l in possibilites:
                        # print(' '.join(l.split()[:self.n - 1])
                        if ' '.join(l.split()[:self.n - 1]) == word_s:
                            candidates.append(l)
                    probs = {i: self.interpolated_probability(
                        i) for i in set(candidates)}
                    if len(probs) == 0:
                        word_s = random.choice(
                            list(self.gram[-2].keys()))
                        continue
                    max_candidate = max(probs, key=probs.get)
                    word_s = ' '.join(max_candidate.split()[1:])
                    if len(word_s.split()) != 0:
                        poetry += word_s.split()[-1] + ' '
                    else:
                        random.choice(list(self.gram[-2].keys()))
                    k+=1
                poetry += '\n'
                word_s = random.choice(list(self.gram[-2].keys()))
            poetry += '\n' if i != 6 else ''
        if self.rev:
            rev_poetry = ''
            for line in poetry.split('\n'):
                if len(line) > 0:
                    rev_poetry += ' '.join(line.split()[::-1])
                    rev_poetry += '\n'
                else:
                    rev_poetry += '\n'
            return rev_poetry
        return poetry

    def generate_verse(self, word_s):
        verse = ''
        verse += word_s + ' '
        word_count = random.randint(6, 9)
        k = 0
        while k < word_count:  # words per verse
            candidates = []
            possibilites = list(self.gram[-1].keys())
            for l in possibilites:
                # print(' '.join(l.split()[:self.n - 1])
                if ' '.join(l.split()[:self.n - 1]) == word_s:
                    candidates.append(l)
            probs = {i: self.interpolated_probability(
                i) for i in set(candidates)}
            if len(probs) == 0:
                word_s = random.choice(
                    list(self.gram[-2].keys()))
                continue
            max_candidate = max(probs, key=probs.get)
            word_s = ' '.join(max_candidate.split()[1:])
            if len(word_s.split()) != 0:
                verse += word_s.split()[-1] + ' '
            else:
                random.choice(list(self.gram[-2].keys()))
            k+=1

        if self.rev:
            return ' '.join(verse.split()[::-1])
        return verse

    def __str__(self):
        lens = []
        for i in range(self.n):
            lens.append(f'\n\t{gram_map[i]}-gram count: {len(self.gram[i])}')
        return f"{gram_map[self.n-1]}-gram model\n\tvocab size: {len(self.vocab)}" + ''.join(lens)


Reading file

In [3]:
with open('poetry.txt', 'r', encoding='utf-8') as f:
    corpus = f.read()
corpus = corpus.split('\n')
corpus = [line.translate(str.maketrans('', '', """،‘۔:%!'’؟""")) for line in corpus if line != '']

back_corpus = [' '.join([str(word) for word in list(tokenizer(line))[-1::-1]]) for line in [line for line in corpus]]

Initializing Models

In [4]:
BigramModel = n_gram_model(2)
BigramModel.train(corpus)

TrigramModel = n_gram_model(3)
TrigramModel.train(corpus)

BackwardBigramModel = n_gram_model(2, rev=True)
BackwardBigramModel.train(back_corpus)

UnigramModel = n_gram_model(1)
UnigramModel.train(corpus)

# Generating poetry via Bigram Model

In [5]:
poetry = BigramModel.generate_poetry()
print(poetry)
print(urdu2roman(poetry))

باندھا گیا تو کیا ہے کہ دستِ نوازش ہو 
عوض فردوس میں ہے کہ صبحِ بہارِ 

پردا نہ ہوا ہے کہ کو میں ہے کہ 
ِخیال میں ہے کہ ناز کا ہے کہ 

اپناہی آشیانہ بنے گا ناپائیدار ہوگا یک 
نفع ہیں کہ ہے کہ خبر 

رشتہٴ گوہر نہیں آتی ہے کہ و دل کا 
مخمور نہیں آتی ہے کہ زندگانیِ شمع ہوں میں 

آمد ہے کہ گے کیا ہے 
پردہ ہے کہ کا ہے کہ داغ 

اے خدا کی طرح عیش نیام سے 
ایراں میں ہے کہ طاعت و دل 

جوشِ قدح سے گزر گیا تو کیا 
پہنچا ہے کہ سرا نہ ہوا ہے 

baandha geia to keia hay keh dsata e noaazash ho 
a'oz frados mein hay keh sbaha e bhaara e 

parada neh hoa hay keh ko mein hay keh 
 eakheiaal mein hay keh naaz ka hay keh 

aapnaahi aasheiaaneh bnay ga naapaaidaar hoga yk 
nfo hein keh hay keh khbar 

rashathٴ gohr nhein aati hay keh wo dl ka 
makhamor nhein aati hay keh zndagaanei e shmo hon mein 

aamad hay keh gay keia hay 
paradeh hay keh ka hay keh daagh 

aaay khda ki trah aeish neiaam say 
aaeiraan mein hay keh taaa't wo dl 

josha e qdah say gzar geia to keia 
phncha hay keh sra neh hoa hay 



# Generating poetry via Backward Bigram Model

In [6]:
poetry = BackwardBigramModel.generate_poetry()
print(poetry)
print(urdu2roman(poetry))

آگ اِس عاشقِ ڈبویا مجھ کو تجھ سے جان جھوٹ
تو ہی کیوں نہ ہو کوچہ دادن

کرے عجب آزاد مرد تھا کچھ روزِ
حق مغفرت کرے عجب آزاد مرد تھا سو اسی نوجواں

کیا ہے تو ہی کیوں نہ مانگ لیتے
ڈبویا مجھ کو تجھ سے کوئی پوچھے بیٹهے

ڈبویا مجھ کو تجھ سے جان جھوٹ
طرز تپاک اہل دنیا جل گیا تائب

ہوں میں ہوں میں ہوں میں ہوں میں دوزخ
و دل و دل تا جگر ودیعتِ مژگانِ چشمِ سوزن

ہوں میں ہوں میں ہوں میں اس اسے شیوہ
کیا ہے کہ مر جائیں گے کیا ہے تیری عظمت

میں ہوں میں ہوں میں ہوں میں اٹھا
ہے تو ہی کیوں نہ ہو جس کے بغیر


aag a eas aaashaqa e ddboeia mjh ko tjh say jaan jhota
to hi keion neh ho kocheh daadn

karay ajab aazaad mrad tha kchh roza ea
haq mghafarat kray ajab aazaad mrad tha so asi nojoaan

keia hay to hi keion neh maang leitay
ddaboeia mjh ko tjh say koi pochhay beitahay

ddaboeia mjh ko tjh say jaan jhota
taraz tpaak ahl dneia jl geia taaiba

hon mein hon mein hon mein hon mein dozakha
o dl wo dl ta jgar wodeia'ta e mzhagaan e chshama e sozn

hon mein hon mein hon mein as asay sheioh
keia hay keh mr jaai

# Generating poetry via Trigram Model

In [7]:
poetry = TrigramModel.generate_poetry()
print(poetry)
print(urdu2roman(poetry))

کے چمکا گمنام تھا وطن میں بازارِ دوست تو 
غازی کا انداز چھٹ جائیں گے کیا کم از گلشنِ 

میں بہر مشتاقاں یار کو مہماں کیے ہوئے نہاں سے 
جو کبھی ملے گا نماز میں کیا خیال ھے 

جو آئے وہ یاں خدا کرے کہ ملے شیخ کو بھی 
پنہاں نکلا گریباں چاک یا دامن یزداں چاک 

تجھے ہوا کیا ہے شب غم بری بلا ہے سرِ 
ز دَامانِ ضمیر افشاندہ داغوں کی بہار نہ پُوچھ ان 

جاں کے لیے ہم مصوّری ہوں گداز وحشت زنداں 
ہوئے ہم بے خودوں کے طاق نسیاں کا آج 

بھی ضرور ہے کتنے ستارے مانند گویا جل 
میں طاقتِ دیدار بھی نہیں رکھتا کی طرف تھے اشارے 

کا تخلص صبور تھا نے میری شکایت کی ہمیں نے طلب 
ہو نگاہ میں شوخی تو دلبری کیا ہے شب غم 

kay chmaka gmnaam tha wotn mein baazaara e dosat to 
ghaazi ka andaaz chht jaain gay keia km az glashn e 

mein bhr mshataaqaan yaar ko mhmaan keiay hoiay nhaan say 
jo kbhi mlay ga nmaaz mein keia kheiaal hay 

jo aaiay woeh yaan khda kray keh mlay sheikh ko bhi 
pnhaan nkala greibaan chaak ya daamn yzadaan chaak 

tajhay hoa keia hay shb ghm bri bla hay sra e 
z dَaamaan e zmeir afashaandeh daag

## Bidirectional Bigram Model class

In [8]:
class bidirectional_bigram_model():
    def __init__(self, BigramModel, BackwardBigramModel):
        self.BigramModel = BigramModel
        self.BackwardBigramModel = BackwardBigramModel

    def generate_poetry(self):
        poetry = ''
        word_s = random.choice(list(self.BigramModel.gram[-2].keys()))
        word_b = ' '.join(word_s.split()[::-1])
        for i in range(7):  # stanzas
            for j in range(2):  # verses
                forward_verse = self.BigramModel.generate_verse(word_s)
                backward_verse = self.BackwardBigramModel.generate_verse(word_b)
                final_verse = backward_verse[3:] + ' ' + forward_verse[:3]
                poetry += final_verse.strip()
                poetry += '\n'
                word_s = random.choice(list(self.BigramModel.gram[-2].keys()))
                word_b = ' '.join(word_s.split()[::-1])
            poetry += '\n' if i != 6 else ''
        return poetry

# Generating poetry via Bidirectional Bigram Model

In [9]:
BidirectionalBigramModel = bidirectional_bigram_model(BigramModel, BackwardBigramModel)
poetry = BidirectionalBigramModel.generate_poetry()
print(poetry)
print(urdu2roman(poetry))

ہے کہ مر جائیں گے کیا ہے کہ بن بن
ہی کیوں نہ ہو جس کے درخور درخ

کے ڈبویا مجھ کو تجھ سے گریز گری
ہوں میں ہوں میں ہوں میں بھی اگر معرکہ معر

جائیں گے کیا ہے اسد غصۂ غصۂ
جائیں گے کیا ہے پردہ چھوڑا چھو

ل و دل و دل کا زوال آمادہ جذبۂ جذب
جائیں گے کیا ہے ملوکیت آثار آثا

یں گے کیا ہے غیر از نگہِ دیدۂِ دید
میں ہوں میں ہوں میں ہوں میں رہیں رہی

ی ڈبویا مجھ کو تجھ سے رُوح اہتــزاز اہت
مر جائیں گے کیا ہے ہر ایک دن گزارا گزا

مر جائیں گے کیا ہے اقبال جیسی فہم فہم
ہی کیوں نہ ہو جس کے در کھلا کھل

hay keh mr jaain gay keia hay keh bn bn
hi keion neh ho js kay drakhor drakha

kay ddboeia mjh ko tjh say greiz grei
hon mein hon mein hon mein bhi agar ma'rakeh ma'ra

jaain gay keia hay asad ghsaa ghsaaa
jaain gay keia hay pradeh chhorra chho

l wo dl wo dl ka zoaal aamaadeh jzabaa jzaba
jaain gay keia hay mlokeit aasaar aasaa

ein gay keia hay gheir az ngh e deidaaa e deida
mein hon mein hon mein hon mein rhein rhei

i ddboeia mjh ko tjh say rُoh ahtaــzaaz ahta
mar jaain gay keia hay hr aeik dn gzaa